In [1]:
"""This code calculated to use a high res. flood depth tiffs
to get wave heights modified 
Written by Taran Kalra, August 19, 2024"""


'This code calculated to use a high res. flood depth tiffs\nto get wave heights modified \nWritten by Taran Kalra, August 19, 2024'

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat              
import rioxarray
import rasterio 
#from create_raster import make_swan_raster_StructuredGrid

In [3]:
cases = ["1perc", "2perc", "5perc", "10perc", "20perc", "50perc"]
years = ["100yr", "50yr", "20year", "10year", "5year", "2year"] # year of return 
max_wave_height = [7.83, 7.43, 7.2, 6.51, 6.11, 5.59]
transformation_factor_restored = 0.5 # Factor that reduces incoming wave height, Based on internal discussions 

In [4]:
scene = "Degraded" # "Restored"

if (scene == "Restored"):
    scenario = "Restored" # "Degraded"
    scenario1= "restored" # degraded"  
    scenario_out = "alt"  # "no_alt"
    max_wave_height = transformation_factor_restored*np.array(max_wave_height)

else:
    scenario = "Degraded"
    scenario1 = "degraded"
    scenario_out = "no_alt"
    #max_wave_height = transformation_factor_degraded*max_wave_height 


# input file names
output_directory = r'C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\final\Wave_height'
flood_depth_directory = r'C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\final\Flood_depths'

In [5]:
breaking_wave_coefficient = 0.78 

In [6]:
# Loop over the filenames in the directory
for i, case in enumerate(cases): 
    outfile = f'{output_directory}\{case}_{scenario_out}.tif' 
 
    flood_depth_file = f'{flood_depth_directory}\{case}.tif'

    # Grid file path specified 
    # this is the finer TIFF file 
    finer_xds = rioxarray.open_rasterio(flood_depth_file) 
    depth = finer_xds.squeeze() 

    # This is different for each pixel 
    #breaking_wave_depth_criteria = 1.3*depth 
    # Reduce the wave height and save it in a temporary array
    #breaking_wave_height = breaking_wave_coefficient * (max_wave_height[i])
    ds_new = depth*0.0 # make an array of zeros 

    modified_max_wave_height = max_wave_height[i] + ds_new # to convert a raster array of max wave height over the domain

    # Calculate the breaking depth criterion
    breaking_criteria = modified_max_wave_height > breaking_wave_coefficient*depth

    # Calculate the new wave height for broken waves
    new_wave_height = breaking_wave_coefficient*depth 


    #print(type(modified_max_wave_height))
    # If depth <= breaking wave depth criteria, then replace the wave height with broken wave heights, otherwise keep it as is. 
    modified_max_wave_height = np.where(breaking_criteria, new_wave_height, modified_max_wave_height)
    # make negative numbers  = 0
    modified_max_wave_height = np.where(modified_max_wave_height<0, 0.0, modified_max_wave_height)
    # Fill the array to get modified wave height 
    ds_new = modified_max_wave_height + ds_new # Add 0's through ds_new to make sure it is a xarray object that can be converted through rio command 
     
    print(type(ds_new))
    print(outfile)
  
    ds_new.rio.to_raster(outfile) # save the new array in a raster before changing the profile of the raster

    print(np.nanmin(ds_new))
    print(np.nanmax(ds_new)) 

<class 'xarray.core.dataarray.DataArray'>
C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\final\Wave_height\1perc_no_alt.tif
0.0
7.83
<class 'xarray.core.dataarray.DataArray'>
C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\final\Wave_height\2perc_no_alt.tif
0.0
7.43
<class 'xarray.core.dataarray.DataArray'>
C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\final\Wave_height\5perc_no_alt.tif
0.0
7.2
<class 'xarray.core.dataarray.DataArray'>
C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\final\Wave_height\10perc_no_alt.tif
0.0
6.51
<class 'xarray.core.dataarray.DataArray'>
C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\final\Wave_height\20perc_no_alt.tif
0.0
6.11
<class 'xarray.core.dataarray.DataArray'>
C:\Users\tkalra\Desktop\NFWP_Maurice_River_NJ\final\Wave_height\50perc_no_alt.tif
0.0
5.59


In [7]:
1.6/2.5

0.64

In [8]:

#fig = plt.figure(figsize=(15,10))
#ax = fig.add_subplot(1,1,1)
#z1_plot = ax.pcolormesh(fine_data.x.values, fine_data.y.values, array, cmap='viridis', vmin=-20, vmax=20)
#plt.colorbar(z1_plot)